# testing stuff

In [180]:
import cv2
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 21,15

BLACK = 0
WHITE = 255

In [12]:
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

In [11]:
def locate_lines(img):
    '''Locate lines.
    Returns: staff lines LOCATIONS and the lines-only image'''
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    method, block, c = cv2.THRESH_BINARY, 51, 35
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    kernel_w, kernel_h = 1,30
    kernel = np.ones((kernel_w, kernel_h), dtype=np.int)
    lines_only_img = cv2.dilate(img_ada, kernel, iterations=1)
    
img = cv2.imread('./images/dataset/muzikanti.JPG', 0) # 0 --> read as grayscale
# locate_lines(img)

In [35]:
##### ## ## ## ## ## ## ##
# Test of a special kernel for line reconstruction (and maybe removal of non-line symbols)
### ## ## ## ## ## ## ##
# Create two kernels. One with a `0` on the X spot, and one with a `1`.
# [
#  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, X, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# ]
kernel0 = np.zeros((11, 11), np.uint8)
kernel0[0] = [1]*11
kernel0[10] = [1]*11

kernel1 = np.zeros((11, 11), np.uint8)
kernel1[0] = [1]*11
kernel1[10] = [1]*11
kernel1[5][5] = 1 # the only difference between `kernel0` and `kernel1`

img = cv2.imread('./images/my_kernel/my_kernel.jpg', 0) # as grayscale
# img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
ret, image_bin = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY) # ret je vrednost praga, image_bin je binarna slika
# plt.imshow(img_bin, 'gray')
print image_bin.dtype
img_er0 = cv2.erode(image_bin, kernel0)
cv2.imwrite('./images/my_kernel/E_0.jpg', img_er0)
img_er1 = cv2.erode(image_bin, kernel1)
cv2.imwrite('./images/my_kernel/E_1.jpg', img_er1)
img_dil0 = cv2.dilate(image_bin, kernel0)
cv2.imwrite('./images/my_kernel/D_0.jpg', img_dil0)
img_dil1 = cv2.dilate(image_bin, kernel1)
cv2.imwrite('./images/my_kernel/D_1.jpg', img_dil1)

uint8


True

In [4]:
img = cv2.imread('./images/dataset/viva1_up.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # konvertovanje iz BGR u GRAYSCALE
# plt.imshow(img, 'gray')  # prikazivanje grayscale slike
cv2.imwrite('./images/dataset/viva_gray.jpg', img)

True

In [29]:
# (unit?) testing of the get_staff method
def run_is_spacing_candidate(run_length, line_spacing):
    return (line_spacing * 0.85 <= run_length <= line_spacing * 1.15)

def run_is_line_candidate(run_length, line_thickness):
    return (line_thickness * 0.4 <= run_length <= line_thickness * 1.6)

In [109]:
# Checks if there is a staff and returns a tuple (found, start_pixel, locations, end_pixel)
#   1.1) True,  or  False
#   1.2) The y-coordinate of the stave's START (start_pixel),  or  -1
#   1.3) Dictionary of pairs: (line_start_y_coordinate, (line_number_in_this_stave, line_thickness))
#   1.4) The y-coordinate of the stave's END (end_pixel),  or  -1
def get_staff(run_index, runs, image_column, cumulative, line_thickness, line_spacing):
#     print ' ~~ get_staff ~~\n  checking', image_column[cumulative:cumulative+150]
    # initialize the return dictionary with a line number 1
    line_locations = {cumulative: (1, runs[run_index])} # first line starts at the current index
    line_counter = 2 # first line is known, look for others
    # cumulative -- how low (Y) are we in this column (image_column) - make_first_row_black_in_runs_calc idea will be discarded
    end_pixel = cumulative + runs[run_index] # determines the y-coordinate where this stave ends
    stave_found = True
    
    for i in range(1, 9):
        run_length = runs[run_index+i]
        if i%2 == 1: # i is odd for line_spacings, since for i=0 we get first LINE's index (run_index)
            if not run_is_spacing_candidate(run_length, line_spacing):
                stave_found = False
                break
        else: # i%2 == 0
            if run_is_line_candidate(run_length, line_thickness): # print 'LINE AT', end_pixel, 'size', run_length
                line_locations[end_pixel] = (line_counter, run_length) # add the line
                line_counter += 1 # we are looking for the next line
        end_pixel += run_length # advance down the column
    
    if not stave_found:
        return False, -1, {}, -1 # not_found, no_start, no_locations, no_end
    
    # prepare the return values
    start_pixel = min(line_locations.keys())
    # get the last line's y_coordinate and add her height to it -- that's where this stave ends
    lowest_line = max(line_locations.keys()) # MAX because the lower the line, the bigger the Y-coordinate
    end_pixel = lowest_line + line_locations[lowest_line][1] # lowest line at `lowest_line`, thickness: line_loc[low_line][1]
    return True, start_pixel, line_locations, end_pixel

In [184]:
# Adds "help" lines and spaces (eng. "ledger lines/spaces") to `locaions` dictionary
def add_helper_spaces_and_lines(locations, line_thickness, line_spacing, cumulative):
    line_thickness = int(round(line_thickness)) # round(2.4) = 2 ||| round(2.6) = 3  :)
    line_spacing = int(round(line_spacing))
    
    help_space_3_up_y = cumulative - (2 * line_thickness) - (3 * line_spacing) 
    locations[help_space_3_up_y] = (0, line_spacing)
    help_line_2_up_y = cumulative - (2 * line_thickness) - (2 * line_spacing)
    locations[help_line_2_up_y] = (1, line_thickness)
    help_space_2_up_y = cumulative - (1 * line_thickness) - (2 * line_spacing)
    locations[help_space_2_up_y] = (2, line_spacing)
    help_line_1_up_y = cumulative - (1 * line_thickness) - (1 * line_spacing)
    locations[help_line_1_up_y] = (3, line_thickness)
    help_space_1_up_y = cumulative - (0 * line_thickness) - (1 * line_spacing)
    locations[help_space_1_up_y] = (4, line_spacing)
    
    lowest_line = max(locations.keys()) # MAX because the lower the LINE, the bigger the Y-coordinate
    lowest_line_end = lowest_line + locations[lowest_line][1]
    
    help_space_1_down_y = lowest_line_end + (0 * line_thickness) + (0 * line_spacing)
    locations[help_space_1_down_y] = (14, line_spacing)
    help_line_1_down_y = lowest_line_end + (0 * line_thickness) + (1 * line_spacing)
    locations[help_line_1_down_y] = (15, line_thickness)
    help_space_2_down_y = lowest_line_end + (1 * line_thickness) + (1 * line_spacing)
    locations[help_space_2_down_y] = (16, line_spacing)
    help_line_2_down_y = lowest_line_end + (1 * line_thickness) + (2 * line_spacing)
    locations[help_line_2_down_y] = (17, line_thickness)
    help_space_3_down_y = lowest_line_end + (2 * line_thickness) + (2 * line_spacing) 
    locations[help_space_3_down_y] = (18, line_spacing)

# Checks if there is a staff and returns a tuple (found, start_pixel, locations, end_pixel)
#   1.1) True,  or  False
#   1.2) The y-coordinate of the stave's START (start_pixel),  or  -1
#   1.3) Dictionary of pairs: (line_or_space_start_y, (line_space_CODE, line_thickness))
#   1.4) The y-coordinate of the stave's END (end_pixel),  or  -1
def get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing):
    print ' ~~ get_staff_with_spaces ~~\n  checking', image_column[cumulative:cumulative+150]
    # initialize the return dictionary with a line number 1, which has the CODE=5. Why 5? See the next line
    # >> help_space3_up (CODE=0), help_line2_up, help_space2_up, help_line1_up, help_space1_up, FIRST_LINE
    locations = {cumulative: (5, runs[run_index])} # first line starts at the current index
    line_space_counter = 6 # first line is saved, now go for other SPACES AND LINES
    # cumulative -> how low (Y) are we in this image column - Make_first_row_black_in_runs_calc idea will be discarded!
    end_pixel = cumulative + runs[run_index] # determines the y-coordinate where this the last-found element end
    stave_found = True
    
    for i in range(1, 9):
        run_length = runs[run_index + i]
        if i%2 == 1: # i is odd for line_spacings, since for i=0 we get first LINE's index (run_index)
            if not run_is_spacing_candidate(run_length, line_spacing):
                stave_found = False
                break
        else: # i%2 == 0
            if not run_is_line_candidate(run_length, line_thickness):
                stave_found = False
                break
        # we came to execute this line, so this run (space/line) is ok
        locations[end_pixel] = (line_space_counter, run_length) # add the SPACE/LINE
        line_space_counter += 1 # look for the next SPACES AND LINES
        end_pixel += run_length # advance down the column
    
    if not stave_found:
        return False, -1, {}, -1 # not_found, no_start, no_locations, no_end
    
    # prepare the return values
    add_helper_spaces_and_lines(locations, line_thickness, line_spacing, cumulative)
    
    start_pixel = min(locations.keys())
    # get the last line's y_coordinate and add her height to it -- that's where this stave ends
    lowest_space = max(locations.keys()) # MAX because the lower the SPACE, the bigger the Y-coordinate
    end_pixel = lowest_space + locations[lowest_space][1] # lowest SPACE at `lowest_space`, height: locations[lowest_space][1]
    return True, start_pixel, locations, end_pixel

In [223]:
def find_staves_in_runs(runs, image_column, line_thickness, line_spacing):
    run_index = 0 # current run index
    cumulative = 0 # how many pixels did we pass - for getting run's color# The paint_the_first_row_black idea will be discarded!!!
    
    staff_counter = 0
    staves = {}
    while (run_index < len(runs)):# WHILE, because we need more control over the index vlue
        run_length = runs[run_index]
        print 'cumulative is now:',cumulative,'| runs[',run_index,'] =',run_length,'| len(runs):',len(runs)
        
        if (image_column[cumulative] == BLACK and run_is_line_candidate(run_length, line_thickness)):
            print 'checking a BLACK run of size', run_length
            
            staff_is_found, start_pixel, staff, end_pixel = get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing)
            if (staff_is_found): # YEA!
                print 'we got them linez! [8)'
                staff_counter += 1
                staves[start_pixel] = (staff_counter, staff, end_pixel)
                # A staff was found, so we need to go down 8 runs until the last line of this staff
                #   0 ,    1   ,   2 ,    3   ,   4 ,    5   ,   6 ,    7   ,  8
                # line, spacing, line, spacing, line, spacing, line, spacing, line
                for r_i in range(run_index, run_index+9): # OR 8??
                    print 'adding', runs[r_i], 'to cumulative'
                    cumulative += runs[r_i]
                run_index += 8
            else: # staff was NOT found
                cumulative += run_length
        else:
            cumulative += run_length
        run_index += 1
    
    return len(staves), staves # return the size and the staves dictionary


In [228]:
run_index = 1 # starting from the first run (the SECOND run is a first-staff-line candidate)
runs = [100, 6,20,5,21,4,20,5,21,6,8] # 100, 106, 126, 131, 152, 156, 176, 181, 202
###### 6 zeros, then twenty times 255 then 5 zeros... [0,0,0,0,0,0, 255,19_more_255, 0,0...]
runs_pixels = [255]*100 + [0]*6 + [255]*20 + [0]*5 + [255]*21 + [0]*4 + [255]*20 + [0]*5 + [255]*21 + [0]*6 + [255]*8
# row_start, black, line_thickness, line_spacing = 23, 5, 4, 20
# row_start, black, line_thickness, line_spacing = 49, 4, 4, 20

# start from X-th pixel, and find lines close to 4px who are separated with around 20px
cumul, thick, spacing = 100, 4, 20

# real line example!
runs_pixels = [0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255]
runs = [5, 517, 4, 16, 4, 16, 3, 17, 4, 16, 4, 294, 4, 15, 4, 16, 4, 16, 4, 15, 4, 289, 4, 16, 4, 15, 4, 16, 3, 16, 4, 286, 5, 15, 4, 15, 4, 16, 4, 15, 4, 1543]
cumul, thick, spacing = 0, 3.5, 15.5
# print 'height of the staff should be:', sum(runs[1:len(runs)-1])
# found, start, locations, end = get_staff_with_spaces(run_index, runs, runs_pixels, cumul, thick, spacing)
#                         (runs, image_column, line_thickness, line_spacing)
staves = find_staves_in_runs(runs, runs_pixels, thick, spacing)
print staves
staves_keylist = staves[1].keys()
staves_keylist.sort()
print '\nSORT BY y-coordinates'
for staves_key in staves_keylist:
    print "STAVE AT Y = %s" % (staves_key)
    staff = staves[1][staves_key][1]
    staff_keylist = staff.keys()
    staff_keylist.sort()
    for staff_key in staff_keylist:
        print '\tstaff_line %s: %s' % (staff_key, staff[staff_key])
    print ''
# print '\n\nSORT BY CODES'
# for key, value in sorted(locations.iteritems(), key=lambda (k,v): (v[0],k)):
#     print "Element %s starts at y = %s" % (value[0], key)
# print '\n--'

cumulative is now: 0 | runs[ 0 ] = 5 | len(runs): 42
checking a BLACK run of size 5
 ~~ get_staff ~~
  checking [0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255]
cumulative is now: 5 | runs[ 1 ] = 517 | len(runs): 42
cumulative is now: 522 | runs[ 2 ] = 4 | len(runs): 42
checking a BLACK run of size 4
 ~~ ge

In [ ]:
def find_staves_in_runs(runs, image_column, line_thickness, line_spacing):
    run_index = 0 # current run index
    cumulative = 0 # how many pixels did we pass - for getting run's color# The paint_the_first_row_black idea will be discarded!!!
    run_length = runs[run_index]
    staff_counter = 0
    staves = {}
    while (run_index < len(runs)):# WHILE, because we need more control over the index vlue
        print 'cumulative is now:', cumulative, '| runs[',run_index,'] =', runs[run_index], '| len(runs):', len(runs)
        # run through runs until a first-staff-line candidate is found
        if not(image_column[cumulative] == BLACK and run_is_line_candidate(run_length, line_thickness)):
            run_index += 1
            if (run_index >= len(runs)):
                break
            cumulative += run_length
            run_length = runs[run_index]
            continue
        
        if run_index >= len(runs):
            break
        print 'checking a run of size', run_length
        # Now, we have a first-staff-line candidate - check for a staff!
        staff_is_found, start_pixel, staff, end_pixel = get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing)
        if (staff_is_found): # YEA!
            print 'we got them linez! [8)'
            staff_counter += 1
            staves[start_pixel] = (staff_counter, staff, end_pixel)
            # A staff was found, so we need to go down 8 runs until the last line of this staff
            #   0 ,    1   ,   2 ,    3   ,   4 ,    5   ,   6 ,    7   ,  8
            # line, spacing, line, spacing, line, spacing, line, spacing, line
            for r_i in range(run_index, run_index+9): # OR 8??
                print 'adding', runs[r_i], 'to cumulative'
                cumulative += runs[r_i]
            run_index += 8
        else:
            ''
            cumulative += run_length
            run_length = runs[run_index]
        
        # we skipped to last line of a found staff if we found it, or didn't go anywhere if we didn't.
        # Now, go on to the next run (it's a spacing, but we need it for `cumulative` calculation)
        run_index += 1
    
    return len(staves), staves # return the size and the staves dictionary


In [229]:
wid = 2400
print wid/5, wid/4, wid/3, wid/2, wid/3*2, wid/4*3 # 480 600 800 1200 1600 1800

480 600 800 1200 1600 1800


### pixel-based staves locating

In [ ]:
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
#     print 'checking', x, 'in', y
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

def find_four_lines_below(column, row_start, black, line_thickness, line_spacing):
    # determine the search position
    # in the given column, from the starting position,
    #   go down for `black` pixels and then go down for line spacing height
    #   then, go back a few pixels, just in case the next line is higher than average
    scaled_spacing = int(line_spacing*0.9) + 1 # <-- might be the cause of all evil :)
    search_position = row_start + black + scaled_spacing
    
    # determine the search thickness we are looking for
    search_for_thickness = 1 if (line_thickness <= 3) else 2  # CAUTION <-- might be the cause of all evil :)
    search_for_array = [0] * search_for_thickness # [0]*5 --> [0, 0, 0, 0, 0]
    
    # determine the search space (exact pixels through which we will search for a line)
    search_height = int(line_thickness)*2 + 1 # thick*2 if (thick <= 3) else thick*1.5 ILI 2.5 ???
    lowest_pixel = search_position + search_height
#     print 'lowest pixel to look at: {} | number of rows: {}'.format(lowest_pixel, len(column))
    if (len(column) > lowest_pixel): # if we have 10 elements, maximum valid index is 9 !
        search_space = column[search_position : lowest_pixel]
        line_below = x_in_y(search_for_array, search_space)
        if (line_below):
            text = 'FOUND a line BELOW\n\tposition: {}|search_thickness: {}|search_height: {}'
#             print text.format(search_position, search_for_thickness, search_height)
            # return True

    # text = 'searching for second line...\n\tposition: {} | search_thickness: {} | search_height: {}'
    # print text.format(search_position, search_for_thickness, search_height)

    # check if there is a line below, by calling the x_in_y function
    second_line = x_in_y(search_for_array, search_space)
    
    if (second_line): print 'WE FOUND A LINE!'
    else: print 'NOT A LINE...'
# end of `find_four_lines_below` method


# THIS DOESN'T HAVE TO RECEIVE `lines_only_img`, CAN IT GET BINARIZED IMG???
def locate_lines___OLD___look_BELOW(lines_only_img, line_thickness, line_spacing):
    '''
    Returns:
        Dictionary of (key, value) pairs where:
          - key is Y coordinate (multiple keys for one detected line),
          - value is a tuple: (staff_number, line_number_in_this_staff)
      '''
    print 'locating_lines....'
    # NO NEED TO CHECK EVERY COLUMN - we can go through only 2,3,4 or 5 (or more)
    # image parts of dimensions: (10px X image_height)  - 10px is experimental value :D
    # for every column      #>>  test - columns 80 to 120
    for col in xrange(80, 120): #>>  lines_only_img.shape[1]
        row = 0 # start from the first pixel in the column
        print '\n| In column', col
        while (row < img.shape[0]):
            black = 0
            # while the pixels
            while (lines_only_img[col][row] == 0):
                black += 1
                # row += 1 # now or as last statement, if we need the starting pixel
            print '\tcounted black pixels: ', black, # A COMMA HERE ! ! ! 
            if ((black > line_thickness * 0.5) and (black < line_thickness * 1.5)):
                print '\tthis black run is a staff line candidate!'
                # check if there are 4 staff lines below this one
                is_first_staff_line, locations =\
                  find_four_lines_below(lines_only_img[0:lines_only_img.shape[0], col]\
                                        , row, black, line_thickness, line_spacing)
                # If there are 4 staff lines, that means WE FOUND A WHOLE STAFF/STAVE!
                # Go downwards in this column for STAFF_HEIGHT.
                # TWO OPTIONS for getting the staff height:
                #   1) NO NEED TO ADD to the `row` variable! Just set it to the `y` coordinate
                #       of the 5th line of the found STAFF/STAVE/ lines system!
                #   
                #   OR
                #   
                #   2) result of:  last line's y_coordinate MINUS first line y_coorindate
                #      
                #      this is a one-liner, heh..
            else: print '\tthis black run is NOT a line candidate!'
            
            print '\tfinishing column', col, '....'
            # row += ?? #
        
    print 'returnign line locations...'
# end of locate_lines

In [230]:
# # # # # # # ########
# testing for methods
# find_four_lines_below(...) and find_lines_up_or_down(...)

# column = [0]*4 + [255]*20 + [0]*5 + [255]*21 + [0]*4 + [255]*20 # [0,0,0,0,255,19_more_255, 0,0...]
# row_start, black, line_thickness, line_spacing = 23, 5, 4, 20
# row_start, black, line_thickness, line_spacing = 49, 4, 4, 20
# find_lines_up_or_down(column, row_start, black, line_thickness, line_spacing)
# row_start, black, line_thickness, line_spacing = 0, 4, 4, 20
# find_four_lines_below(column, row_start, black, line_thickness, line_spacing)

## Second approach in generating the lines-only image  --==> (*more robust*) 

In [ ]:
# Removes any black runs from the input image that are bigger
# than the average line_thickness * multiplier (currently = 1.7)
def rm_everything_but_lines(lines_only_img):
    img_height, img_width = lines_only_img.shape # unpack values
    
    # runs calculation, for thickness and spacing
    runs, black_runs_flat, white_runs_flat = calculate_runs(lines_only_img)
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_height)
    
    staff_thickness_multiplier = 1.7 # 1.6, 1.8 ??
    
    clear_img = lines_only_img.copy()
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is deleted/whitened
                if (run_length > line_thickness * staff_thickness_multiplier):
                    clear_img[cumulative:cumulative + run_length, c] = [255]*(run_length)
            cumulative += run_length
    
    #%% % % % % % % % % % % %%
    #  R E C O N S T R U C T   T H E   I M A G E  with  MY_KERNEL !!!! !!   <<< GREAT idea! >>>
    #%% % % % % % % % % % % %%
    
    # dilate with 1x KW --> erode with 2x KW   ||||||  ==~~--- BAD IDEA CURRENTLY ~~--=== |||||
    kernel_w, kernel_h = 1, 71
    dilated = dilate_and_save(clear_img, kernel_w, kernel_h)
    kernel_w_e, kernel_h_e = 2, 71
    clear_lines_only_img = erode_and_save(dilated, kernel_w, kernel_h) # same kernel as dilation
    
    return clear_lines_only_img

def get_lines_only_img_CLEAR(img):
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    method, block, c = cv2.THRESH_BINARY, 55, 9
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    # dilate with 1x50 --> erode with 2x50  | this has the best results
    kernel_w, kernel_h = 1, 50
    dilated = dilate_and_save(img_ada, kernel_w, kernel_h)
    kernel_w_e, kernel_h_e = 2, 50
    lines_only_img = erode_and_save(dilated, kernel_w, kernel_h) # same kernel as dilation
    
    clear_lines_only_img = rm_everything_but_lines(lines_only_img)
    
    return clear_lines_only_img

## Old line-removal algorithms

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# ALG 2 - look at white runs around the line candidate
# 2*d . . . 1*d . . . X . . . 1*d . . . 2*d
#  0         1        -        2         3
# In order for X to be whitened (erased):
#

def spacing_is_ok(run_length, line_spacing):
#     print '>>> spacing_is_ok <<< run_length: {}, line_spacing: {}'.format(run_length, line_spacing)
    return (run_length < (line_spacing * 1.15)) and (run_length > (line_spacing * 0.85))

def find_lines_up_or_down(column, row_start, black, line_thickness, line_spacing):
#     print 'column', column
#     print 'row_start', row_start, 'black', black, 'line_thickness', line_thickness, 'line_spacing', line_spacing
    scaled_spacing = int(line_spacing*0.9) + 1 # <-- might be the cause of all evil :)
    search_position = row_start + black + scaled_spacing
    # determine the search thickness we are looking for
    search_for_thickness = 1 if (line_thickness <= 3) else 2  # CAUTION <-- might be the cause of all evil :)
    search_for_array = [0] * search_for_thickness # [0]*5 --> [0, 0, 0, 0, 0]
    # determine the search space (exact pixels through which we will search for a line)
    search_height = int(line_thickness)*2 + 1 # thick*2 if (thick <= 3) else thick*1.5 ILI 2.5 ???
    lowest_pixel = search_position + search_height
#     print 'lowest pixel to look at: {} | number of rows: {}'.format(lowest_pixel, len(column))
    if (len(column) > lowest_pixel): # if we have 10 elements, maximum valid index is 9 !
        search_space = column[search_position : lowest_pixel]
        line_below = x_in_y(search_for_array, search_space)
        if (line_below):
            text = 'FOUND a line BELOW\n\tposition: {}|search_thickness: {}|search_height: {}'
#             print text.format(search_position, search_for_thickness, search_height)
            return True
    
    search_position = row_start - scaled_spacing - search_height
#     print 'search position for lines above:', search_position
    lowest_pixel = row_start - scaled_spacing
    if (search_position >= 0):
        search_space = column[search_position : lowest_pixel]
        line_above = x_in_y(search_for_array, search_space)
        if line_above:
            text = 'Found a line ABOVE\n\tposition: {}|search_thickness: {}|search_height: {}'
#             print text.format(search_position, search_for_thickness, search_height)
            return True
    
    return False
# end of find_lines_up_or_down

def run_has_lines_up_or_down(runs_in_column, current_index, line_thickness, line_spacing):
    '''check if there is a line on `line_spacing` above or below the current index '''
    lines = [255,255,255,255]
    return 1

def rm_staff_lines_up_down_neighbours(img, runs, line_thickness, line_spacing, staff_thickness_multiplier):
    '''Removes staff lines by looking at every black run's neighbours, above and below.'''
    # Original image is being changed + Python passes by obj-ref, so it's necessary
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            deleted = False # did we delete the run
            if (r % 2 == 0): # check every black run, shorter than `thickness * multiplier`
                if (run_length < line_thickness * staff_thickness_multiplier):
                    # this is not the last run === check the run AFTER this one
                    if (r + 1 < (len(runs[c]))):
                        if spacing_is_ok(runs[c][r+1], line_spacing): # we found a line - delete it
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
                        # else: # more conditions to add.. like..
                                # check if there is a black run above or below the current one,
                                # on distance that is equal to line_spacing +-1,
                                # whose lenght is = line_thickness +-1
                            
                    # this is not the first run === check the run BEFORE this one
                    if (not deleted and (r - 1 >= 0)):
                        if spacing_is_ok(runs[c][r-1], line_spacing): # we found a line - delete it
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
                    # run_length > 1  ILI_DA_BUDE cumulative > 0  ? ? ? ?? ? ? ? ?? ?
                    if (not deleted): # and run_length > 1)
                        column = img[0:img.shape[0], c].tolist() # convert numpy array to python list
                        if find_lines_up_or_down(column, cumulative, run_length, line_thickness, line_spacing):
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
#                     if run_has_lines_up_or_down(runs[c], r, line_thickness, line_spacing)):
#                 else: # run is thicker than the average line
#                     if (run_length < line_thickness * 2.2): # ASSUMPTION --- for empty note-heads
                        
            cumulative += run_length # !!! # we advanced down for `run_length` pixels !
    return p

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# ALG 3 - compare neighbour points on distance `d`,
# to the left and right, from the observed point X:
# 2*d . . . 1*d . . . X . . . 1*d . . . 2*d
#  0         1        -        2         3
# In order for X to be whitened (erased):
#   - points[1] and points[2] should be black, or
#   - points[0] and points[1] should be black, or
#   - points[2] and points[3] should be black.

def initialize_points(c, distance, img, compare_point, runs):
    '''Gets the neighbours located '''
    points = [255,255,255,255] # initially, pixels are white (maybe -1 if not accessible)
    # 2*left , 1*left , 1*right, 2*right
    if (c - distance >= 0): # we can get the 1*left pixel
        points[1] = img[compare_point, c - distance]
        if (c - 2*distance >= 0): # we can get the 2*left pixel
            points[0] = img[compare_point, c - 2*distance]
    if (c + distance < len(runs)): # we can get the right pixel
        points[2] = img[compare_point, c + distance]
        if (c + 2*distance < len(runs)):
            points[3] = img[compare_point, c + 2*distance]
    return points

def rm_staff_lines_side_neighbours(img, runs, thickness, spacing, thickness_mul, distance, lines_only_img = []):
    if distance > len(runs)/2:
        print 'WHOA! Distance is: {} and there are only {} columns'.format(distance, len(runs))
        return img
    int_thickness = int(thickness)+1
    p = img.copy() # we will erase someimg lines, so copy the image
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    if (lines_only_img != []): # check neighbours on the lines_only image, if we passed it
        img = lines_only_img
    for c in xrange(len(runs)): # for every column
        cumulative = 0 # number of passed pixels
        for r in xrange(len(runs[c])):
            run_length = runs[c][r]
            if (r % 2 == 0):# for every black run
#                 if (run_length < thickness * thickness_mul): # ADD THIS BACK ! ! ! !! !
#                     p[cumulative:cumulative + run_length, c] = [255]*(run_length)
#                     cumulative += run_length # !!!
#                     continue
#                 else:
                pixels_to_remove = int_thickness # <=== this goes to ELSE
                # we need to check if we passed the image height!
                compare_point = min(img.shape[0]-1, cumulative + pixels_to_remove/2) # + run_length/2 # maybe later
                # 4 values of neighbour pixels, some of them must be black,
                #   for deletion of the observed pixel to happen.
                points = initialize_points(c, distance, img, compare_point, runs)
                if (points[1] == 0):
                    if (points[2] == 0):
                        p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
                    else: # out of bounds or white # !! !!  ASSUMPTION !!  !!!
                    # if (points[2] == -1): # more robust? Needs points=[-1,-1,-1,-1]
                        # two to the left is black?
                        if (points[0] == 0):
                            p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
                        # else: NIJE LINIJA :D
                # no black point on the left
                else:
                    if (points[2] == 0 and points[3] == 0):
                        p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
            cumulative += run_length # !!!
    # return the new, processed image
    return p

In [231]:
# param values
threshold_types = [cv2.ADAPTIVE_THRESH_MEAN_C]# , cv2.ADAPTIVE_THRESH_GAUSSIAN_C # const values are 0 and 1
block_sizes = [33,35,37,39,43]# [11, 19, 27, 35, 43] # [11,15,19,23,27,31,35, 39, 43]
c_values = [29,33,35,37] # [17,19,21] # [11, 19, 27, 35, 43] # [11,15,19,23,27,31,35, 39, 43]
staff_thickness_multipliers = [1.3, 1.5] # [1., 1.5, 2., 2.5]

for thresh_type in threshold_types:
    for block_size in block_sizes:
        for c_val in c_values:
            for s_t_mul in staff_thickness_multipliers:
                parse_image(img, thresh_type, block_size, c_val, s_t_mul)

NameError: name 'parse_image' is not defined